Demand Forecaster from https://www.kaggle.com/c/demand-forecasting-kernels-only

In [2]:
## Imports

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch as torch
import torch.nn as nn

In [3]:
## Import data

path_in = './data/'

train = pd.read_csv(path_in + 'train.csv')
test = pd.read_csv(path_in + 'test.csv')